In [ ]:
import re
import string
import urllib.parse as urlparse


HTTP_METHODS = {
    "GET": "GET(.|\n)+?(?=GET|POST|\Z)",
    "POST": "POST(.|\n)+?(?=GET|POST|\Z)"
}

URL_REGEX = "http.+?(?= )"
BODY_REGEX = "(?<=\n\n).+(?=\n\n)"



def parse(path, request_reg: string):
    with open(path) as file:
        data = file.read()

        requests = []
        [requests.append(request.group(0)) for request in re.finditer(request_reg, data, re.MULTILINE)]

        return requests

def parseParamsFromUrl(request):
    url = parseUrl(request)
    return urlparse.parse_qs(urlparse.urlparse(url).query)

def parseUrl(request):
    return re.search(URL_REGEX, request).group(0)

def parseParamsFromBody(request):
    params = {}
    body = re.search(BODY_REGEX, request)

    if body is not None:
        body = body.group(0)

        params = urlparse.parse_qs(body)

    return params

In [4]:
from pathlib import Path

In [7]:
class FileNotFound(Exception):
    def __init__(self, message: str = "File not found", *args: object) -> None:
        if len(args) == 0:
            raise TypeError("FileNotFound exception requires exactly one argument.")
        elif len(args) > 1:
            raise TypeError("FileNotFound exception requires exactly one argument, but multiple were given.")
        elif not (isinstance(args[0], str) or isinstance(args[0], Path)):
            raise TypeError("FileNotFound exception requires a string or Path argument representing the file path.")
        self.filepath = args[0]
        super().__init__(message)
     
    def __str__(self) -> str:
        return super().__str__() + f"\nFile '{self.filepath}' not found - please check the filepath and try again."

In [8]:
# Example usage oof my class
try:
    raise FileNotFound("Custom message that involves losses files by the transmitted path", Path("path/to/none__existent/file.txt"))
except FileNotFound as e:
    print(e)

Custom message that involves losses files by the transmitted path
File 'path\to\none__existent\file.txt' not found - please check the filepath and try again.


In [9]:
try:
    raise FileNotFound("Custom message that involves losses files by the transmitted path", "path/to/none__existent/file.txt")
except FileNotFound as e:
    print(e)

Custom message that involves losses files by the transmitted path
File 'path/to/none__existent/file.txt' not found - please check the filepath and try again.


In [41]:
from abc import ABC, abstractmethod

class AbstractBaseException(Exception, ABC):
    """
    Abstract base class for custom exceptions.
    """

    def __init__(self, message: str, *args: object) -> None:
        self.message = message
        Exception.__init__(self, message)
        

    @abstractmethod
    def __str__(self) -> str:
        """
        Subclasses must implement this method.
        """
        return f"AbstractBaseException: {self.message}"

# Example of a concrete exception subclass
class BaseError(AbstractBaseException):
    def __init__(self, message: str, *args: object) -> None:
        AbstractBaseException.__init__(self, message, *args)

    def __str__(self) -> str:
        return f"{self.__class__.__name__} exception: {self.message}" + super().__init__() + f'\nDEBUG INFO aboat MRO: {BaseError.mro()}'

# Example of using the exceptions
try:
    raise BaseError("Not include in module currecnt directory -> /path/to/file")
except AbstractBaseException as e:
    print(e)  # This will call the __str__ method of FileNotFound

TypeError: __init__() missing 1 required positional argument: 'message'

In [42]:
from abc import ABC, abstractmethod

class AbstractBaseException(Exception, ABC):
    """
    Abstract base class for custom exceptions.
    """

    def __init__(self, message: str, *args: object) -> None:
        self.message = message
        super().__init__(message)
        

    @abstractmethod
    def __str__(self) -> str:
        """
        Subclasses must implement this method.
        """
        pass  # Abstract method, to be implemented by subclasses

# Example of a concrete exception subclass
class BaseError(AbstractBaseException):
    def __init__(self, message: str, *args: object) -> None:
        super().__init__(message, *args)

    def __str__(self) -> str:
        return f"{self.__class__.__name__} exception: {self.message}" + f'\nDEBUG INFO about MRO: {BaseError.mro()}'

# Example of using the exceptions
try:
    raise BaseError("Not included in the current directory -> /path/to/file")
except AbstractBaseException as e:
    print(e)  # This will call the __str__ method of BaseError

BaseError exception: Not included in the current directory -> /path/to/file
DEBUG INFO about MRO: [<class '__main__.BaseError'>, <class '__main__.AbstractBaseException'>, <class 'Exception'>, <class 'BaseException'>, <class 'abc.ABC'>, <class 'object'>]


In [47]:
from dataclasses import dataclass, field

@dataclass
class RequestDatframe:
    dataframe_id: int
    requests: list = field(default_factory=list)

In [49]:
RequestDatframe(0, []) == RequestDatframe(0, []),\
    RequestDatframe(0, [1,2,3]) == RequestDatframe(0, [1,2,3])

(True, True)

In [72]:
from typing import Dict, Any, Tuple
@dataclass
class RequestDatframeMod:
    requests: list = field(default_factory=list)
    columns: List[str] = field(default_factory=list)
    indexes: List[int] = field(default_factory=list)
    attrs: Dict[str, Any] = field(default_factory=dict)
    dataframe_id: int = -1
    ignore_errors: bool = True
    # ignore init attributes
    dtype: object = field(init = False)
    shape: Tuple[int, ...] = field(init= False)
    
    
    
    # init repr eq
    # after exec
    # __post_init__
    
    def __post_init__(self):
        self.shape = len(self.requests), len(self.columns)
        if not self.requests:
            if not self.ignore_errors: raise TypeError("self.requests cannot be empty")
            self.dtype = None
        else:
            self.dtype = type(self.requests[0])
        # для нубов могу объяснить
        # декоратор (_ for _ in ())
        # у декораторов можно выбросить исключение  через метод throw
        # когда пытаемся получить значение из генератора который пуст
        # дальше вызов анонимной (лямбда) функции на лету (сразу)
        # self.dtype = type(self.requests) if self.requests else (lambda: (_ for _ in ()).throw(TypeError("self.requests cannot be empty")))()


In [73]:
RequestDatframeMod().__annotations__, \
RequestDatframeMod().__dict__, \
    RequestDatframeMod()

TypeError: tuple expected at most 1 arguments, got 2

In [10]:
from tqdm import trange, tqdm, tqdm_notebook

In [28]:
class FileFormatIsWrong(Exception):
    def __init__(self, *args: object) -> None:
        if len(args) == 0:
            raise TypeError(f"{self.__class__.__name__} exception requires exactly one argument.")
        elif len(args) > 1:
            raise TypeError(f"{self.__class__.__name__} requires exactly one argument, but multiple were given.")
        elif not (isinstance(args[0], str) or isinstance(args[0], Path)):
            raise TypeError(f"{self.__class__.__name__} exception requires a string or Path argument representing the file path.")
        self.message = args[0]
        super().__init__(self.message)
     
    def __str__(self) -> str:
        return super().__str__() + f"\nFile Format Is Wrong. Request must need start from GET|POST|PUT"

In [29]:
try:
    raise FileFormatIsWrong("Custom message that means problem with file format")
except FileFormatIsWrong as e:
    print(e)

Custom message that means problem with file format
File Format Is Wrong. Request must need start from GET|POST|PUT


In [15]:
from typing import List
ENCODING = 'windows-1252' # Spanish letters charset
SHIFT = Path('../..')
TRAIN_DATA_PATH = SHIFT / "normalTrafficTraining.txt"
TEST_DATA_NORMAL_PATH = SHIFT / "normalTrafficTest.txt"
TEST_DATA_ANOMALY_PATH = SHIFT / "anomalousTrafficTest.txt"
def read_file(file_path: Path) -> str:
    if not file_path.exists():
        raise FileNotFound("read file error", file_path)
    
        
    with file_path.open('r', encoding=ENCODING) as file:
        return file.read()

def starts_with_method(line: str) -> bool:
    return line.startswith(('GET', 'POST', 'PUT'))
   
def parse(path: Path, request_reg: str = SPLIT_REQUEST) -> List[str]:
    request_readed_from_txt = read_file(path)
    requests = re.split(request_reg, request_readed_from_txt)
    rejected_indexes = []
    for index, request in tqdm(enumerate(requests), total=len(requests)):
        if not starts_with_method(request):
            rejected_indexes.append(index)
    ass
    return requests

In [16]:
rs = parse(TRAIN_DATA_PATH)
print(rs[0])

GET http://localhost:8080/tienda1/index.jsp HTTP/1.1
User-Agent: Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)
Pragma: no-cache
Cache-control: no-cache
Accept: text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5
Accept-Encoding: x-gzip, x-deflate, gzip, deflate
Accept-Charset: utf-8, utf-8;q=0.5, *;q=0.5
Accept-Language: en
Host: localhost:8080
Cookie: JSESSIONID=1F767F17239C9B670A39E9B10C3825F4
Connection: close



In [ ]:
FileNotFound('message: File not found, )

In [ ]:
SPLIT_REQUEST =  r'\n\n(?=GET|POST)'
def parse(path, request_reg: string):
    with open(path) as file:
        data = file.read()

        requests = []
        [requests.append(request.group(0)) for request in re.finditer(request_reg, data, re.MULTILINE)]

        return requests

In [12]:
import re, regex
import string
from urllib.parse import unquote, unquote_plus, parse_qs, parse_qsl
from typing import List
import typing_extensions
from pathlib import Path


ENCODING = 'windows-1252' # Spanish letters charset
SHIFT = Path('../..')
TRAIN_DATA_PATH = SHIFT / "normalTrafficTraining.txt"
TEST_DATA_NORMAL_PATH = SHIFT / "normalTrafficTest.txt"
TEST_DATA_ANOMALY_PATH = SHIFT / "anomalousTrafficTest.txt"
ANOMALIES_LABEL = -1
NORMAL_LABEL = 1
SPLIT_REQUEST = r'\n\n(?=GET|POST)'


class FileNotFound(Exception):
    def __init__(self, message: str = "File not found", *args: object) -> None:
        if len(args) == 0:
            raise TypeError("FileNotFound exception requires exactly one argument.")
        elif len(args) > 1:
            raise TypeError("FileNotFound exception requires exactly one argument, but multiple were given.")
        elif not isinstance(args[0], str):
            raise TypeError("FileNotFound exception requires a string argument representing the file path.")
    
        super().__init__(message, *args)
    
    def __str__(self) -> str:
        return super().__str__() + f"\nFile '{self.args[0]}' not found - please check the filepath and try again."




class FileFormatIsWrong(Exception):
    pass


class RequestsReader:
    def __init__(self, file_path: Path) -> None:
        if not file_path.exists():
        pass
    
    def read_file(file_path: Path) -> str:
        with file_path.open('r', encoding=ENCODING) as file:
            return file.read()
        
    def parse(path: Path, request_reg: str = SPLIT_REQUEST) -> List[str]:
        request_readed_from_txt = read_file(path)
        requests = re.split(request_reg, request_readed_from_txt)
        return requests

    def parseParamsFromUrl(request):
        url = parseUrl(request)
        return urlparse.parse_qs(urlparse.urlparse(url).query)

    def parseUrl(request):
        return re.search(URL_REGEX, request).group(0)

    def parseParamsFromBody(request):
        params = {}
        body = re.search(BODY_REGEX, request)

        if body is not None:
            body = body.group(0)

            params = urlparse.parse_qs(body)

        return params


IndentationError: expected an indented block (1020359585.py, line 43)